In [1]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [2]:
!pip install -q "summac==0.0.4" --no-deps


In [3]:
import types, sys

# 给 transformers.models.deprecated.ernie_m 做一个假的空模块
dummy_module = types.ModuleType("transformers.models.deprecated.ernie_m")
sys.modules["transformers.models.deprecated.ernie_m"] = dummy_module

In [ ]:
from huggingface_hub import login
login("your_huggingface_token_here")


In [5]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM
import torch, numpy as np

print("numpy:", np.__version__)

MODEL_ID = "mercuryujia/bart-large-multi-news"

# 1. 修改 config，让 early_stopping 不为 None
config = AutoConfig.from_pretrained(MODEL_ID)
config.early_stopping = True

# 2. 慢版 tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)

# 3. 使用修改过的 config 加载模型
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID, config=config)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


numpy: 2.0.2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/bart/configuration_bart.py:177: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


generation_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        

In [6]:
from transformers import GenerationConfig

source_document = """
President Joe Biden met with congressional leaders at the White House on Tuesday
to discuss a new economic relief package. The meeting comes as inflation remains
high and lawmakers are divided over the size of government spending.

Meanwhile, the Federal Reserve signaled it may slow down the pace of interest
rate hikes later this year if price increases continue to cool.

Economists say the negotiations in Washington could have major consequences
for financial markets and for American households.
"""

K = 4

gen_config = GenerationConfig(
    bos_token_id=0,
    decoder_start_token_id=2,
    early_stopping=True,
    eos_token_id=[2],
    forced_bos_token_id=0,
    forced_eos_token_id=2,
    length_penalty=2.0,
    max_length=142,
    min_length=56,
    no_repeat_ngram_size=3,
    num_beams=4,
    pad_token_id=1,
)

inputs = tokenizer(
    source_document,
    max_length=1024,
    truncation=True,
    return_tensors="pt"
).to(device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        generation_config=gen_config,
        num_return_sequences=K,
    )

candidate_summaries = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

print("=== 生成的候选摘要 ===")
for i, s in enumerate(candidate_summaries):
    print(f"[Cand {i}] {s}\n")

=== 生成的候选摘要 ===
[Cand 0] – Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the AP. The meeting comes as inflation remains high and lawmakers are divided over the size of government spending. Meanwhile, the Federal Reserve signaled it may slow down the pace of interest rate hikes later this year if price increases continue to cool.

[Cand 1] – Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the Hill. The meeting comes as inflation remains high and lawmakers are divided over the size of government spending. Meanwhile, the Federal Reserve signaled it may slow down the pace of interest rate hikes later this year if price increases continue to cool.

[Cand 2] – Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the AP. The meeting comes as inflation remains high, and lawmakers are divided over

In [7]:
import types, sys
from transformers import PretrainedConfig

# Create a dummy module for transformers.models.deprecated.ernie_m
dummy_module = types.ModuleType("transformers.models.deprecated.ernie_m")

class ErnieMConfig(PretrainedConfig):
    model_type = "ernie_m"

# Expose ErnieMConfig in that dummy module
setattr(dummy_module, "ErnieMConfig", ErnieMConfig)

# Register the dummy module so that AutoConfig / AutoModel can find it
sys.modules["transformers.models.deprecated.ernie_m"] = dummy_module




Semantic Alignment Score

In [8]:

from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Use a sentence-embedding model from Hugging Face
EMB_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

emb_tokenizer = AutoTokenizer.from_pretrained(EMB_MODEL_NAME)
emb_model = AutoModel.from_pretrained(EMB_MODEL_NAME).to(device)

def embed_sentences(sent_list):
    """
    Encode a list of sentences into L2-normalized embeddings using mean pooling.
    """
    if len(sent_list) == 0:
        return None

    inputs = emb_tokenizer(
        sent_list,
        padding=True,
        truncation=True,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        outputs = emb_model(**inputs)
        last_hidden = outputs.last_hidden_state  # [batch, seq, hidden]
        attention_mask = inputs["attention_mask"]  # [batch, seq]

        # Mean pooling
        mask = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
        sum_embeddings = (last_hidden * mask).sum(dim=1)
        sum_mask = mask.sum(dim=1)
        embeddings = sum_embeddings / sum_mask

        # L2 normalize
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

    return embeddings  # [batch, hidden]


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [9]:
def semantic_alignment_score(doc: str, summary: str) -> float:
    """
    Compute a semantic alignment score between a source document and a summary.
    For each summary sentence, we take the maximum cosine similarity to any
    source sentence, then average over all summary sentences.

    Higher score means the summary is more semantically aligned with the source.
    """
    # 1. Sentence split
    doc_sents = [s.strip() for s in nltk.sent_tokenize(doc) if s.strip()]
    sum_sents = [s.strip() for s in nltk.sent_tokenize(summary) if s.strip()]

    if not doc_sents or not sum_sents:
        return 0.0

    # 2. Encode sentences
    doc_embs = embed_sentences(doc_sents)    # [N_doc, dim]
    sum_embs = embed_sentences(sum_sents)    # [N_sum, dim]

    if doc_embs is None or sum_embs is None:
        return 0.0

    # 3. Cosine similarity matrix: [N_sum, N_doc]
    # Since embeddings are L2-normalized, dot product = cosine similarity
    cos_sim = torch.matmul(sum_embs, doc_embs.T)  # [N_sum, N_doc]

    # 4. For each summary sentence, take max similarity to any source sentence
    max_per_sum_sent, _ = torch.max(cos_sim, dim=1)  # [N_sum]

    # 5. Average over summary sentences
    score = float(max_per_sum_sent.mean().cpu().item())
    return score


In [10]:
semantic_scores = [
    semantic_alignment_score(source_document, s)
    for s in candidate_summaries
]

print("=== Semantic alignment scores (higher is better) ===")
for i, (s, sc) in enumerate(zip(candidate_summaries, semantic_scores)):
    print(f"[Cand {i}] score = {sc:.4f}")
    print(s)
    print("-" * 80)


=== Semantic alignment scores (higher is better) ===
[Cand 0] score = 0.9865
– Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the AP. The meeting comes as inflation remains high and lawmakers are divided over the size of government spending. Meanwhile, the Federal Reserve signaled it may slow down the pace of interest rate hikes later this year if price increases continue to cool.
--------------------------------------------------------------------------------
[Cand 1] score = 0.9886
– Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the Hill. The meeting comes as inflation remains high and lawmakers are divided over the size of government spending. Meanwhile, the Federal Reserve signaled it may slow down the pace of interest rate hikes later this year if price increases continue to cool.
----------------------------------------------------------------

In [11]:
!pip install -q datasets

from datasets import load_dataset

# 载入 parquet 版 Multi-News
multi_news = load_dataset("Awesome075/multi_news_parquet")

print(multi_news)
print("Columns:", multi_news["test"].column_names)
print("Example 0:", multi_news["test"][0])

README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/323M [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/39.5M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/40.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})
Columns: ['document', 'summary']
Example 0: {'document': 'GOP Eyes Gains As Voters In 11 States Pick Governors \n \n Enlarge this image toggle caption Jim Cole/AP Jim Cole/AP \n \n Voters in 11 states will pick their governors tonight, and Republicans appear on track to increase their numbers by at least one, with the potential to extend their hold to more than two-thirds of the nation\'s top state offices. \n \n Eight of the gubernatorial seats up for grabs are now held by Democrats; three are in Republican hands. Republicans currently hold 29 governorships, Democrats have 20, and Rhode Island\'s Gov. Lincoln Chafee is an Independent. \n \n Polls and race analysts suggest that only thre

In [12]:
import torch

def generate_bart_summary(source_text: str, max_new_tokens: int = 128) -> str:
    """
    Use the finetuned BART model to generate a single summary for the source_text.
    """
    model.eval()
    inputs = tokenizer(
        source_text,
        return_tensors="pt",
        truncation=True,
        max_length=1024,   # Multi-News 原文很长，这里截断一下
    ).to(device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=4,
            early_stopping=True,
            length_penalty=2.0,        # 👈 关键：显式给一个数，不用 None
            no_repeat_ngram_size=3,    # 可选：防止重复
        )

    summary = tokenizer.decode(
        output_ids[0],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True,
    )
    return summary.strip()



Score after fine tuned

In [13]:
import torch
from tqdm import tqdm

# 设定实验参数
test_beam_sizes = [16, 32, 64]
# 设定最大样本数 (只跑50条)
MAX_SAMPLES = 50
split_name = "test"

dataset_split = multi_news[split_name]

# 存储结果的列表
experiment_results = []

print(f"开始实验，测试 Beam Sizes: {test_beam_sizes}")

for k in test_beam_sizes:
    print(f"\n=== Running Generation with Beam Size = {k} ===")

    current_beam_preds = []

    # 使用 enumerate 来计数，方便在 50 条时停止
    for i, item in tqdm(enumerate(dataset_split), total=MAX_SAMPLES):
        if i >= MAX_SAMPLES:
            break

        # --- 1. 获取原始文本 (自动尝试常见的列名) ---
        # 不同的数据集列名不同，这里做一个自动匹配
        if 'document' in item:
            text = item['document'] # XSum 常用
        elif 'article' in item:
            text = item['article']  # CNN/DailyMail 常用
        elif 'text' in item:
            text = item['text']
        elif 'source' in item:
            text = item['source']
        else:
            # 如果找不到，打印一下列名帮助调试
            print(f"报错：找不到文本列。你的列名是: {item.keys()}")
            break

        # --- 2. 现场分词 (Tokenization) ---
        # 将文本转换为模型需要的 input_ids
        inputs = tokenizer(
            text,
            return_tensors="pt",
            max_length=1024,   # 输入文章太长时截断
            truncation=True
        ).to(device)

        # --- 3. 生成摘要 ---
        with torch.no_grad():
            outputs = model.generate(
                inputs["input_ids"],
                max_length=128,      # 摘要最大长度 (保持不变)
                num_beams=k,         # <--- 实验变量：Beam Size
                early_stopping=True,
                no_repeat_ngram_size=3, # 防止重复，通常建议加上
                length_penalty=1.0
            )

        # --- 4. 解码 ---
        decoded_pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        current_beam_preds.append(decoded_pred)

    # 存储当前 Beam Size 的所有结果
    experiment_results.append({
        "beam_size": k,
        "predictions": current_beam_preds
    })

    print(f"Beam {k} 完成。生成了 {len(current_beam_preds)} 条摘要。")

print("\n所有实验运行结束！")

开始实验，测试 Beam Sizes: [16, 32, 64]

=== Running Generation with Beam Size = 16 ===


  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:1733: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
100%|██████████| 50/50 [05:24<00:00,  6.50s/it]


Beam 16 完成。生成了 50 条摘要。

=== Running Generation with Beam Size = 32 ===


100%|██████████| 50/50 [08:29<00:00, 10.19s/it]


Beam 32 完成。生成了 50 条摘要。

=== Running Generation with Beam Size = 64 ===


100%|██████████| 50/50 [16:32<00:00, 19.85s/it]

Beam 64 完成。生成了 50 条摘要。

所有实验运行结束！


In [14]:
from sentence_transformers import CrossEncoder
import numpy as np
import torch

# 检查设备
device = "cuda" if torch.cuda.is_available() else "cpu"

# 加载 SAS 模型 (通常使用在 STS 数据集上微调过的模型)
# 推荐模型: "cross-encoder/stsb-roberta-large" 或 "cross-encoder/stsb-distilroberta-base" (速度更快)
sas_model = CrossEncoder("cross-encoder/stsb-roberta-large", device=device)

def calculate_sas(sources, generated_summaries, batch_size=32):
    """
    计算 SAS 分数
    :param sources: 原文列表 (List[str]) 或 参考摘要列表
    :param generated_summaries: 模型生成的摘要列表 (List[str])
    :return: 平均 SAS 分数 (float)
    """
    # 构造模型需要的配对输入: [(原文1, 生成1), (原文2, 生成2), ...]
    # 注意：SAS通常是计算 "生成摘要" 与 "参考摘要(Gold Summary)" 的相似度
    # 如果你没有参考摘要，也可以计算与 "原文" 的相似度，但标准SAS是对比 Gold Summary。

    pairs = list(zip(sources, generated_summaries))

    # 模型预测 (返回 0-1 之间的分数)
    print(f"正在计算 SAS 分数 (共 {len(pairs)} 条)...")
    scores = sas_model.predict(pairs, batch_size=batch_size, show_progress_bar=True)

    return np.mean(scores), scores

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [15]:
# 1. 准备参考摘要列表
reference_summaries = []
MAX_SAMPLES = 50  # 必须和你生成时的数量保持一致

print("正在提取参考摘要...")

# 遍历数据集提取参考摘要
for i, item in enumerate(dataset_split):
    if i >= MAX_SAMPLES:
        break

    # 尝试寻找参考摘要的列名 (自动适配不同数据集)
    # CNN/DM通常是'highlights', XSum是'summary'
    if 'summary' in item:
        ref = item['summary']
    elif 'highlights' in item:
        ref = item['highlights']
    elif 'target' in item:
        ref = item['target']
    elif 'label' in item:
        ref = item['label']
    else:
        # 如果实在找不到，可能需要你手动指定列名
        print(f"Error: 找不到参考摘要的列名，现有的列名: {item.keys()}")
        break

    reference_summaries.append(ref)

print(f"提取完成，共 {len(reference_summaries)} 条参考摘要。")

正在提取参考摘要...
提取完成，共 50 条参考摘要。


In [16]:
# 2. 遍历实验结果进行打分
# 假设 experiment_results 是你之前生成阶段保存的列表
# 结构应该是: [{'beam_size': 1, 'predictions': [...]}, {'beam_size': 2, 'predictions': [...]}, ...]

final_scores_data = []

print(f"\n开始对 {len(experiment_results)} 组实验结果进行 SAS 打分...\n")

for result in experiment_results:
    k = result['beam_size']
    preds = result['predictions']

    # 安全检查：确保生成数量和参考数量一致
    if len(preds) != len(reference_summaries):
        print(f"警告：Beam Size {k} 生成了 {len(preds)} 条，但参考摘要有 {len(reference_summaries)} 条。跳过此组。")
        continue

    print(f"===正在评估 Beam Size = {k} ===")

    # 调用你的 SAS 计算函数
    mean_sas, all_scores = calculate_sas(reference_summaries, preds)

    print(f"Result: Beam Size {k} -> Mean SAS: {mean_sas:.4f}\n")

    final_scores_data.append({
        "beam_size": k,
        "mean_sas": mean_sas,
        "all_scores": all_scores # 保存详细分数以便后续分析
    })

print("所有打分完成！")


开始对 3 组实验结果进行 SAS 打分...

===正在评估 Beam Size = 16 ===
正在计算 SAS 分数 (共 50 条)...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Result: Beam Size 16 -> Mean SAS: 0.4764

===正在评估 Beam Size = 32 ===
正在计算 SAS 分数 (共 50 条)...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Result: Beam Size 32 -> Mean SAS: 0.4716

===正在评估 Beam Size = 64 ===
正在计算 SAS 分数 (共 50 条)...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Result: Beam Size 64 -> Mean SAS: 0.4653

所有打分完成！
